Для выполнения этого эксперимента, давайте начнем с загрузки данных, реализации логистического оракула и методов оптимизации (градиентного спуска и Ньютона). Затем мы можем провести эксперименты и построить необходимые графики.

In [ ]:
import numpy as np
from scipy.special import expit
from scipy.sparse import load_svmlight_file
from sklearn.datasets import load_svmlight_file as load_svmlight_file_sklearn
import matplotlib.pyplot as plt
import time

# Загрузка данных и конвертация их в формат CSR матрицы
def load_and_convert_data(file_path):
    data = load_svmlight_file_sklearn(file_path)
    return data[0], data[1]

# Логистический оракул
class LogisticOracle:
    def __init__(self, A, b, reg_coef=1.0):
        self.A = A
        self.b = b
        self.reg_coef = reg_coef

    def func(self, x):
        logits = self.A.dot(x)
        loss = np.sum(np.log(1 + np.exp(-self.b * logits)))
        reg_term = 0.5 * self.reg_coef * np.linalg.norm(x) ** 2
        return loss + reg_term

    def grad(self, x):
        logits = self.A.dot(x)
        sigmoid_vals = expit(-self.b * logits)
        gradient = -self.A.T.dot(self.b * sigmoid_vals)
        reg_term = self.reg_coef * x
        return gradient + reg_term

    def hess(self, x):
        logits = self.A.dot(x)
        sigmoid_vals = expit(self.b * logits)
        diag_vals = sigmoid_vals * (1 - sigmoid_vals)
        hessian = self.A.T.dot(diags(self.b ** 2 * diag_vals, 0)).dot(self.A)
        hessian += self.reg_coef * np.eye(len(x))
        return hessian

# Градиентный спуск
def gradient_descent(oracle, x_0, learning_rate=0.1, tolerance=1e-5, max_iter=1000):
    x_k = np.copy(x_0)
    history = {'time': [], 'func': [], 'grad_norm': []}
    start_time = time.time()

    for iteration in range(max_iter):
        gradient = oracle.grad(x_k)
        grad_norm = np.linalg.norm(gradient)
        history['time'].append(time.time() - start_time)
        history['func'].append(oracle.func(x_k))
        history['grad_norm'].append(grad_norm)

        if grad_norm < tolerance:
            break

        x_k -= learning_rate * gradient

    return x_k, history

# Ньютоновский метод
def newton(oracle, x_0, tolerance=1e-5, max_iter=1000):
    x_k = np.copy(x_0)
    history = {'time': [], 'func': [], 'grad_norm': []}
    start_time = time.time()

    for iteration in range(max_iter):
        gradient = oracle.grad(x_k)
        grad_norm = np.linalg.norm(gradient)
        history['time'].append(time.time() - start_time)
        history['func'].append(oracle.func(x_k))
        history['grad_norm'].append(grad_norm)

        if grad_norm < tolerance:
            break

        hessian = oracle.hess(x_k)
        try:
            step = np.linalg.solve(hessian, -gradient)
        except np.linalg.LinAlgError:
            # Неудачное решение системы линейных уравнений - выходим из цикла
            break

        x_k += step

    return x_k, history

# Загрузка данных и создание оракула
data_file_paths = ['w8a', 'gisette', 'real-sim']
for data_file_path in data_file_paths:
    A, b = load_and_convert_data(data_file_path)
    reg_coef = 1.0 / len(b)  # Коэффициент регуляризации
    oracle = LogisticOracle(A, b, reg_coef)

    # Градиентный спуск
    x_gd, history_gd = gradient_descent(oracle, np.zeros(A.shape[1]), tolerance=1e-5, max_iter=1000)

    # Ньютоновский метод
    x_newton, history_newton = newton(oracle, np.zeros(A.shape[1]), tolerance=1e-5, max_iter=1000)

    # Графики сходимости
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history_gd['time'], history_gd['func'], label='Градиентный спуск')
    plt.plot(history_newton['time'], history_newton['func'], label='Ньютоновский метод')
    plt.xlabel('Время, сек')
    plt.ylabel('Значение функции')
    plt.legend()
    plt.title(f'Сходимость на наборе данных: {data_file_path}')

    plt.subplot(1, 2, 2)
    plt.semilogy(history_gd['time'], np.array(history_gd['grad_norm']) ** 2 / (history_gd['grad_norm'][0] ** 2),
                 label='Градиентный спуск')
    plt.semilogy(history_newton['time'], np.array(history_newton['grad_norm']) ** 2 / (history_newton['grad_norm'][0] ** 2),
                 label='Ньютоновский метод')
    plt.xlabel('Время, сек')
    plt.ylabel('Относительный квадрат нормы градиента')
    plt.legend()
    plt.title(f'Относительный квадрат нормы градиента на наборе данных: {data_file_path}')

    plt.tight_layout()
    plt.show()


Этот код загрузит каждый из трех наборов данных, запустит градиентный спуск и метод Ньютона на каждом из них, а затем визуализирует результаты в виде графиков функции и нормы градиента от времени работы методов. Пожалуйста, убедитесь, что у вас есть папка data в том же каталоге, что и этот скрипт, и что в ней находятся файлы w8a, gisette и real-sim с соответствующими данными.
Код загружает каждый из трех наборов данных, запускает градиентный спуск и метод Ньютона.